In [1]:
pwd

u'/home/simengy/Notebook'

In [2]:
cd /home/simengy/Data\ Science/KDD2015

/home/simengy/Data Science/KDD2015


### Loading data

In [3]:
import pandas as pd

enroll_train = pd.read_csv('data/train/enrollment_train.csv', header=False)
log_train = pd.read_csv('data/train/log_train.csv', header=False)
label_train = pd.read_csv('data/train/truth_train.csv', header=False)

cource_hirach = pd.read_csv('data/object.csv', header=False)

In [4]:
print enroll_train.shape
print log_train.shape
print label_train.shape

(120542, 3)
(8157277, 5)
(120541, 2)


In [6]:
for col in enroll_train.columns:

    levels = pd.Categorical(enroll_train[col]).categories
    print col, levels.shape

enrollment_id (120542,)
username (79186,)
course_id (39,)


In [34]:
for col in log_train.columns:

    levels = pd.Categorical(log_train[col]).levels
    print col, levels.shape

enrollment_id (120542,)
time (3922167,)
source (2,)
event (7,)
object (5890,)


In [33]:
for col in cource_hirach.columns:

    levels = pd.Categorical(cource_hirach[col]).levels
    print col, levels.shape

course_id (39,)
module_id (26750,)
category (15,)
children (13192,)
start (421,)


In [7]:
log_train.loc[1:100]

label_train['0'].mean()

0.79293352469284306

### Feature engineering

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()

for col in enroll_train.columns:

    levels = pd.Categorical(enroll_train[col]).levels
    print col, levels.shape

    
print 
for col in log_train.columns:

    levels = pd.Categorical(log_train[col]).levels
    print col, levels.shape
    
    if col == 'event':
        print enc.fit_transform(log_train[col].T.to_dict().values())    
    if col == 'object':
        print enc.fit_transform(log_train[col].T.to_dict().values())
        
        
        
    if col == 'sources':
        
        pd.core.reshape.get_dummies(log_train[col])
    

        

### Feature selection

### Model training

In [48]:
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score, train_test_split

import sys
sys.path.append('/home/simengy/git/xgboost/wrapper/')
import xgboost as xgb


y_columns = [name for name in train_with_labels.columns if name.startswith('y')]

X_numerical_base, X_numerical_meta, X_sparse_base, X_sparse_meta, y_base, y_meta = train_test_split(
    X_numerical, 
    X_sparse, 
    train_with_labels[y_columns].values,
    test_size = 0.5
)

X_meta = [] 
X_test_meta = []

print "Build meta"


param = {'bst:max_depth':8, 'bst:eta':0.2, 'silent':0, 'objective':'binary:logistic' }
        param['nthread'] = 16
        plst = param.items()
        #plst += [('eval_metric', 'auc')] # Multiple evals can be handled in this way
        plst += [('eval_metric', 'logloss')]
        num_round = 100
        
        dX_base = xgb.DMatrix(X_numerical_base, label = y)
        bst = xgb.train(plst, dX_base, num_round)
        
        dX_num_meta = xgb.DMatrix(X_numerical_meta)
        X_meta.append(bst.predict(dX_num_meta))
        
        dX_test_num = xgb.DMatrix(X_test_numerical)
        X_test_meta.append(bst.predict(dX_test_num))
        
        print i, 'xgboost = ', datetime.now() - t1
        t2 = datetime.now()
        
        logit = LogisticRegression(C=0.01, tol=0.000001)
        logit.fit(X_sparse_base, y)
        X_meta.append(logit.predict_proba(X_sparse_meta))
        X_test_meta.append(logit.predict_proba(X_test_sparse))
        
        print i, 'logit = ', datetime.now() - t2
        
X_meta = np.column_stack(X_meta)
X_test_meta = np.column_stack(X_test_meta)
